In [2]:
# day 4 크롤링 관련입니다. 
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import os

In [3]:
r = requests.get("http://www.wooribank.com/",
                timeout =15,
                headers = {
                    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"
                })
r.encoding = "utf-8" # encoding 방식 지정 중요!

r.status_code # 200 대이면 정상입니다. 

200

In [4]:
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.title)
print(soup.title.string)

<title>우리은행</title>
우리은행


In [5]:
soup.select("div[class='tit-5 absolute js-display-hover']")

[<div class="tit-5 absolute js-display-hover">
 <a class="js-display-hover-trigger" href="https://spot.wooribank.com/pot/Dream?withyou=po"><img alt="금융상품" src="https://simg.wooribank.com/img/intro/header/btn_tit_4_intro_new_211130.png"/></a>
 <div class="js-display-hover-area level2">
 <span class="arrow"></span>
 <ul class="menu">
 <li class="title"><a href="https://spot.wooribank.com/pot/Dream?withyou=po">금융상품</a></li>
 <li><a href="https://spot.wooribank.com/pot/Dream?withyou=PORMG0002">추천상품</a></li>
 <li><a href="https://spot.wooribank.com/pot/Dream?withyou=PODEP0001">예금</a></li>
 <li><a href="https://spot.wooribank.com/pot/Dream?withyou=OWFDM0003">펀드</a></li>
 <li><a href="https://spot.wooribank.com/pot/Dream?withyou=ln">대출</a></li>
 <li><a href="https://spot.wooribank.com/pot/Dream?withyou=fx">외환</a></li>
 <li><a href="https://spot.wooribank.com/pot/Dream?withyou=POGLD0001">골드</a></li>
 <li><a href="https://spot.wooribank.com/pot/Dream?withyou=POTRT0001">신탁</a></li>
 <li><a href=

In [6]:
for item in soup.select("div[class='tit-5 absolute js-display-hover'] li"):
    print(item.text) # select_one은 한개 == find , select = find_all

금융상품
추천상품
예금
펀드
대출
외환
골드
신탁
보험
퇴직연금
시니어플러스 은퇴설계
ISA


In [7]:
# 실습문제 풀어보자 
# 오늘의 코스피 받아오기 

r = requests.get("https://finance.naver.com/")
r.encoding = "utf-8" # encoding 방식 지정 중요!

r.status_code # 200 대이면 정상입니다. 

#soup.select("div[class='tit-5 absolute js-display-hover']")
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.title.string)
kospi = soup.select_one("span[class='num']")
print(kospi.string)

네이버 금융
2,604.24


In [8]:
# finance date 크롤링 
!pip install finance-datareader
import FinanceDataReader as fdr
import re #정규식 부분입니다. 

In [10]:
#투자 지표 수집이 가능합니다. 
fdr.DataReader('AAPL','2019').head(3)  # 주의 쌍따움표는 안됩니다. 왜 안되지?
#fdr.DataReader('AAPL', '2017').head(3)

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2019-01-02,39.48,38.72,39.71,38.56,148160000.0,0.0010
2019-01-03,35.55,35.99,36.43,35.50,365250000.0,-0.0995
2019-01-04,37.06,36.13,37.14,35.95,234430000.0,0.0425


In [11]:
# 기간 설정도 간으합니다.
fdr.DataReader('AMAZ','2019/01/01','2019/12/31').head(3) #기간을 해보는 것 - / 붙여쓰기 다 가능합니다. 
# USD/KRW 환율입니다. 


,Close,Open,High,Low,Volume,Change
Date,,,,,,
2019-01-02,0.21,0.2,0.21,0.20,6400.0,0.1667
2019-01-03,0.18,0.2,0.20,0.18,5720.0,-0.1429
2019-01-04,0.20,0.2,0.20,0.20,12670.0,0.1111


In [12]:
stocks = fdr.StockListing('krx') # KRX krx 대소문자 구별 안함 
stocks.head(5)
stocks.dropna(inplace=True) # 지표가 없는 것들 삭제 전처리 중

In [13]:
base_url = "https://search.itooza.com/search.htm?seName={}&jl=&search_ck=&sm=&sd=2021-08-09&ed=2021-09-08&ds_de=&page=&cpv="
formatted_url = base_url.format('060310')# S3 의 주식 symbol {} 안에 들어 갈것 동적으로 입력합니다. 
html = requests.get(formatted_url).content

soup = BeautifulSoup(html,'lxml')




In [ ]:
#beautifulsoup 파싱 방법, html.parser 설치 필요없고 중간 속, lxml 속도가 빠름 추가 설치 필요
# html5lib 웹브라우저와 같은 방식으로 페이지 파싱 합니다. 느립니다. 

In [24]:
for code, name in stocks[:5][['Symbol','Name']].values: #[행][열] 여러개 담을때는 리스트 형태로 입력해 줘야 합니다. 
    print(code, name) # 테스트 용도로 출력
    while True:
        try:
            url = base_url.format(code)
            html = requests.get(url).content
            soup = BeautifulSoup(html,'lxml')
            break
        except:
            print('error')
            time.sleep(15)
            
#div[cl"ass='[it-5s='ex']"olute js-display-hover']
    table = soup.select("table[class='ex']")
    #table = soup.find("table",{"class":"ex"}) # find와 select 차이 점이 무엇일까?
    
    print(table)
    if table == None: # 없으면 할일이 없다 넘어감 
        continue
    
    idx = ["2021-2Q", "2021-1Q",
           "2020-4Q", "2020-3Q", "2020-2Q", "2020-1Q",
           "2019-4Q", "2019-3Q", "2019-2Q", "2019-1Q",
           "2018-4Q", "2018-3Q"] # 인덱스 정의 
    
    tbody = table.find('tbody')
    t_row = tbody.select('tr')
    
    result_dic ={}
    for tr in t_row:
        col_name = tr.select_one('th').text
        values = [td.text for td in tr.select('td')]
        result_dic[col_name] = values
    
    result  = pd.DataFrame.from_dict(result_dic)
    print(result)
    result.drop('주가',axis=1,inplace=True) # 주가열 필요없
    result['분기'] = idx
    
    if code == '060310': print(result) # 첫 루프를 돌 때만 result 출력

    result.to_csv(name + ".csv", index = False) # csv 파일로 저장. 파일 목록 확인해보기
    
    time.sleep(1) # 1초씩 재우기
    
    

060310 3S
[<table border="1" class="ex">
<thead>
<tr>
<th>투자지표</th>
<th>	21.12월</th>
<th>	21.09월</th>
<th>	21.06월</th>
<th>	21.03월</th>
<th>	20.12월</th>
<th>	20.09월</th>
<th>	20.06월</th>
<th>	20.03월</th>
<th>	19.12월</th>
<th>	19.09월</th>
<th>	19.06월</th>
<th>	19.03월</th>
</tr>
</thead>
<tbody>
<tr>
<th>주당순이익(EPS,연결지배)</th>
<td>80</td>
<td>61</td>
<td>18</td>
<td>15</td>
<td>-65</td>
<td>-62</td>
<td>-52</td>
<td>-24</td>
<td>60</td>
<td>80</td>
<td>76</td>
<td class="last">32</td>
</tr>
<tr>
<th>주당순이익(EPS,개별)</th>
<td>79</td>
<td>59</td>
<td>22</td>
<td>17</td>
<td>-57</td>
<td>-54</td>
<td>-44</td>
<td>-16</td>
<td>53</td>
<td>75</td>
<td>70</td>
<td class="last">26</td>
</tr>
<tr>
<th><abbr title="(주가 / EPS)">PER</abbr> (배)</th>
<td>42.09</td>
<td>67.92</td>
<td>180.27</td>
<td>155.73</td>
<td>N/A</td>
<td>N/A</td>
<td>N/A</td>
<td>N/A</td>
<td>47.04</td>
<td>45.97</td>
<td>35.07</td>
<td class="last">75.83</td>
</tr>
<tr>
<th>주당순자산(지분법)</th>
<td>848</td>
<td>825</td>
<td>797</td>
<t

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?